<a href="https://colab.research.google.com/github/Niweera/LearnDataAnalytics/blob/master/Intro_to_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 1 - ML Fundamentals



## AI

The effort to automate intellectual tasks normally performed by humans.



### ML

ML < AI

Rather than giving the program the rules, an algorithm finds the rules for us.



## NN

NN < ML < AI

Neural Network is a form of machine learning that uses a layered representation of data.




## Different Types of ML

1. Unsupervised Learning
2. Supervised Learning
3. Reinforcement Learning

# Module 2 - Intro to Tensorflow


Install Tensorflow

In [0]:
pip install tensorflow

Importing TensorFlow

In [0]:
%tensorflow_version 2.x # this line is required if you are in a notebook


In [0]:
import tensorflow as tf
print(tf.version)

Creating Tensors



In [0]:
string = tf.Variable("this is a string",tf.string) # these are scalars
number = tf.Variable(324, tf.int32)
floating = tf.Variable(3.14, tf.float64)

Rank/Degree of Tensors

In [0]:
rank0_tensor = tf.Variable("Test",tf.string)
rank1_tensor = tf.Variable(["Test"],tf.string)
rank2_tensor = tf.Variable([["test","ok"],["test","yes"]],tf.string)

Determining the rank of a tensor

In [0]:
tf.rank(rank2_tensor)

<tf.Tensor: shape=(), dtype=int32, numpy=2>

Shape of a tensor

In [0]:
rank2_tensor.shape

TensorShape([2, 2])

Changing shape of a tensor

In [0]:
tensor1 = tf.ones([1,2,3])
tensor2 = tf.reshape(tensor1,[2,3,1])
tensor3 = tf.reshape(tensor2,[3,-1])

print(tensor1)
print()
print(tensor2)
print()
print(tensor3)

# Module 3 - Core Learning Algorithms

 ## Linear Regression

In [0]:
!pip install -q sklearn
%tensorflow_version 2.x

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

Load dataset

In [0]:
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv') # training data
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv') # testing data
y_train = dftrain.pop('survived') # removes the survived column from the df and stores it in the given variable
y_eval = dfeval.pop('survived')

View data frames

In [0]:
dftrain.head()

In [0]:
dfeval.head()

Set up feature columns

In [0]:
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique()  # gets a list of all unique values from given feature column
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))



In [0]:
print(feature_columns)

Input function

In [0]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():  # inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle(1000)  # randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
    return ds  # return a batch of the dataset
  return input_function  # return a function object for use

train_input_fn = make_input_fn(dftrain, y_train)  # here we will call the input_function that was returned to us to get a dataset object we can feed to the model
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

Creating the model

In [0]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)
# We create a linear estimtor by passing the feature columns we created earlier

Training the model

In [0]:
linear_est.train(train_input_fn)  # train
result = linear_est.evaluate(eval_input_fn)  # get model metrics/stats by testing on tetsing data

clear_output()  # clears console output
print(result['accuracy'])  # the result variable is simply a dict of stats about our model

0.77272725


Prediction

In [0]:
pred_dicts = list(linear_est.predict(eval_input_fn))
clear_output()

In [0]:
dfeval.loc[0]

sex                          male
age                            35
n_siblings_spouses              0
parch                           0
fare                         8.05
class                       Third
deck                      unknown
embark_town           Southampton
alone                           y
Name: 0, dtype: object

In [0]:
y_eval.loc[0]

0

In [0]:
pred_dicts[0]

{'all_class_ids': array([0, 1], dtype=int32),
 'all_classes': array([b'0', b'1'], dtype=object),
 'class_ids': array([0]),
 'classes': array([b'0'], dtype=object),
 'logistic': array([0.06463394], dtype=float32),
 'logits': array([-2.6721983], dtype=float32),
 'probabilities': array([0.9353661 , 0.06463394], dtype=float32)}

In [0]:
list(pred_dicts[0]["probabilities"])

[0.9353661, 0.06463394]

## Classification

In [0]:
!pip install -q sklearn
%tensorflow_version 2.x

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

Loading dataset

In [0]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [0]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
# Here we use keras (a module inside of TensorFlow) to grab our datasets and read them into a pandas dataframe

In [0]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


Seperate y axis data

In [0]:
y_train = train.pop("Species")
y_test = test.pop("Species")

In [0]:
y_train.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

Input function

In [0]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

Feature Columns

In [0]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

Building the model

In [0]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=3)

Training the model

In [0]:
classifier.train(
    input_fn=lambda: input_fn(train, y_train, training=True),
    steps=5000)
# We include a lambda to avoid creating an inner function previously

Evaluation the model

In [0]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, y_test, training=False))


clear_output()
eval_result["accuracy"]

0.93333334

Predictions

In [0]:
def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

In [0]:
features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}



In [0]:
print("Please type numeric values as prompted.")
for feature in features:
  valid = True
  while valid: 
    val = input(feature + ": ")
    if not val.isdigit(): valid = False

  predict[feature] = [float(val)]

In [0]:
predictions = classifier.predict(input_fn=lambda: input_fn(predict))

class_ids = [the predicted class id]
probabilities = [class 0 probability, class 1 probability, class 2 probability]


In [0]:
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    clear_output()
    print(f'{SPECIES[class_id]} with the probability of {100 * probability}%')


Virginica with the probability of 81.41812682151794%


## Clustering


## Hidden Markov models

To create a hidden markov model we need.

- States
- Observation Distribution
- Transition Distribution

Setup

In [0]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook

In [0]:
import tensorflow_probability as tfp  # We are using a different module from tensorflow this time
import tensorflow as tf

Weather model

We will model a simple weather system and try to predict the temperature on each day given the following information.
1. Cold days are encoded by a 0 and hot days are encoded by a 1.
2. The first day in our sequence has an 80% chance of being cold.
3. A cold day has a 30% chance of being followed by a hot day.
4. A hot day has a 20% chance of being followed by a cold day.
5. On each day the temperature is
 normally distributed with mean and standard deviation 0 and 5 on
 a cold day and mean and standard deviation 15 and 10 on a hot day.

In [0]:
tfd = tfp.distributions  # making a shortcut for later on
initial_distribution = tfd.Categorical(probs=[0.2, 0.8])  # Refer to point 2 above
transition_distribution = tfd.Categorical(probs=[[0.5, 0.5],
                                                 [0.2, 0.8]])  # refer to points 3 and 4 above
observation_distribution = tfd.Normal(loc=[0., 15.], scale=[5., 10.])  # refer to point 5 above

# the loc argument represents the mean and the scale is the standard devitation

Create hidden markov model

In [0]:
model = tfd.HiddenMarkovModel(
    initial_distribution=initial_distribution,
    transition_distribution=transition_distribution,
    observation_distribution=observation_distribution,
    num_steps=7)

In [0]:
mean = model.mean()

# due to the way TensorFlow works on a lower level we need to evaluate part of the graph
# from within a session to see the value of this tensor

# in the new version of tensorflow we need to use tf.compat.v1.Session() rather than just tf.Session()
with tf.compat.v1.Session() as sess:  
  print(mean.numpy())

[12.       11.1      10.83     10.748999 10.724699 10.71741  10.715222]


# Module 4 - NN with Tensorflow